In [87]:
import numpy as np
import os
from PIL import Image
from collections import defaultdict
import sklearn.metrics
import pandas as pd


In [88]:
def tensor_unfold(tensor, mode):
    x, y, z = tensor.shape
    if mode == 3:
        return np.concatenate([tensor[i, :, :].T for i in range(x)], axis=1)
    elif mode == 2:
        return np.concatenate([tensor[:, :, i].T for i in range(z)], axis=1)    
    else:
        return np.concatenate([tensor[:, i, :] for i in range(y)], axis=1)    
    
    
def tensor_fold(dims,mat, mode):
    tensor = np.empty(dims)
    x, y, z = dims
    q, r = mat.shape
    if mode==3:
        sect = range(0, r + 1, y)
        for i in range(x):
            a, b = sect[i], sect[i + 1]
            tensor[i, :, :] = mat[:, a : b].T
    elif mode == 2:
        sect = range(0, r + 1, x)
        for i in range(z):
            a,b = sect[i], sect[i + 1]
            tensor[:, :, i] = mat[:, a : b].T
    else:
        sect = range(0, r + 1, z)
        for i in range(y):
            a, b = sect[i], sect[i + 1]
            tensor[:, i, :] = mat[:, a : b]             
    return tensor      

def tensor_multiplication(tensor, matrix, mode):
    x, y, z = tensor.shape
    q, r = matrix.shape
    if mode == 3:
        mat = np.empty((q, x * y))
        mat = np.concatenate([np.dot(matrix, tensor[i, :, :].T) for i in range(x)], axis=1)
        return tensor_fold( (x, y, q),mat, mode) 
    elif mode == 2:
        mat = np.empty((q, x * z))
        mat = np.concatenate([np.dot(matrix, tensor[:, :, i].T) for i in range(z)], axis=1)
        return tensor_fold((x, q, z),mat,  mode)  
    else:
        mat = np.empty((q, y * z))
        mat = np.concatenate([np.dot(matrix, tensor[:, i, :]) for i in range(y)], axis=1)
        return tensor_fold((q, y, z),mat, mode) 


In [89]:
def HOSVD(images_tensor):
    unfold1 = tensor_unfold(images_tensor, mode=1)
    unfold2 = tensor_unfold(images_tensor, mode=2)
    unfold3 = tensor_unfold(images_tensor, mode=3)

    u1, s1, v1 = np.linalg.svd(unfold1, full_matrices=False)
    u2, s2, v2 = np.linalg.svd(unfold2, full_matrices=False)
    u3, s3, v3 = np.linalg.svd(unfold3, full_matrices=False)
    imu1 = tensor_multiplication(images_tensor, u1.T, mode=1)
    imu2 = tensor_multiplication(imu1, u2.T, mode=2)
    s = tensor_multiplication(imu2, u3.T, mode=3)
    return s, u1, u2, u3   

In [90]:
def retrieve(typee):
    if typee == 'train':
        images = defaultdict(list)
        for i in os.listdir(f'E:/MSc1/Computational Data mining/CDM_Assignment_8/YALE_faces/{typee}/'):
            subject_name = int(i.split('.')[0][-2:])
            image = Image.open(f'E:/MSc1/Computational Data mining/CDM_Assignment_8/YALE_faces/{typee}/{i}')
            image_array = np.array(image).flatten('F')
            images[subject_name].append(image_array)
        return images        
    else:
        images = defaultdict(list)
        subject_name = 0
        for i in os.listdir(f'E:/MSc1/Computational Data mining/CDM_Assignment_8/YALE_faces/{typee}/'):
            subject_name += 1
            image = Image.open(f'E:/MSc1/Computational Data mining/CDM_Assignment_8/YALE_faces/{typee}/{i}')
            image_array = np.array(image).flatten('F')
            images[subject_name].append(image_array)
        return images

In [91]:
images = retrieve('train')
tensor = []
for key, value in images.items() :
    stacked = np.stack(value, axis=1)
    tensor.append(stacked)   
A = np.stack(tensor, axis=0)

transposed_array = np.transpose(A, (1, 0, 2))
reshaped_array = np.reshape(transposed_array, (A.shape[1], A.shape[0], -1))                    
A = np.transpose(reshaped_array, (0,2,1)) 

In [92]:
A.shape
tensor

[array([[130, 130, 130, ..., 130, 130, 130],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 203, ..., 255, 255, 255],
        [255, 255, 215, ..., 255, 255, 255],
        [ 68,  68,  68, ...,  68,  68,  68]], dtype=uint8),
 array([[130, 109, 130, ..., 108, 109, 110],
        [255, 235, 255, ..., 234, 236, 238],
        [255, 253, 255, ..., 251, 252, 253],
        ...,
        [255, 183,  60, ..., 176, 182, 182],
        [255, 179,  59, ..., 171, 175, 175],
        [ 68,  68,  68, ...,  68,  68,  68]], dtype=uint8),
 array([[130, 130, 130, ..., 130, 130, 130],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255,  25, ..., 255, 255, 255],
        [255, 255,  25, ..., 255, 255, 255],
        [ 68,  68,  68, ...,  68,  68,  68]], dtype=uint8),
 array([[130, 130, 130, ..., 130, 130, 130],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 25

In [93]:
S,U1,U2,U3 = HOSVD(A)
first = tensor_multiplication (S, U1, mode=1)
sec = tensor_multiplication (first, U2, mode = 2)

In [94]:
test = retrieve('test')
distances = defaultdict(list)
for key, value in test.items():
    if key != 0:
        a = []
        for i in range(10):
            x, residuals, rank, singular_values = np.linalg.lstsq(sec[:, i, :], np.array(value[0]), rcond=None)
            a.append(x)
        for j in range(14):
            distances[key].append(min(np.linalg.norm(np.array(a) - U3.T[:,j], axis = 1))) # you need just min

In [95]:
distances

defaultdict(list,
            {1: [0.192500285112985,
              1.1115301661896861,
              0.9350952191374029,
              0.9154129425875634,
              0.9882063588962341,
              1.031361974318151,
              1.1169767853403587,
              0.999565774949422,
              0.8762018789907068,
              0.974879537328189,
              1.0483272185550383,
              0.8068730149143366,
              1.1110554553345517,
              1.0704015420202966],
             2: [0.8262562027059082,
              0.5071142014921893,
              1.1069976678487614,
              0.8170824429033774,
              0.8210054650153737,
              1.1382876750645592,
              1.0039981491799812,
              0.977259918131889,
              1.0578191302636302,
              0.9808412250822784,
              1.067693573843939,
              0.6735859113263936,
              1.0578692274705335,
              1.1240841491478089],
             3: [1.101387371

In [96]:
min_index = defaultdict(list)
for key, value in distances.items():
    if min(value) <= 0.5:
        min_index[key] = value.index(min(value))
    else: 
        min_index[key] = -1

In [97]:
labels=[]
for i in range(14):
    labels.append(i)
for i in range (11):
    labels.append(-1)
pred_labels = list(min_index.values())
sklearn.metrics.accuracy_score(labels,pred_labels)

0.84